In [1]:
import os, glob
from osgeo import gdal
import cv2
import numpy as np
import scipy
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# path to your working directory
workdir = 'path/to/your/workdir'
# path to wherever your data is stored, if different from workdir
datadir = 'path/to/your/datadir'

# load in all SLC dates
dates = [os.path.basename(x) for x in glob.glob(f'{datadir}/VV/merged/SLC/2*')]
dates = sorted(dates)
nd = len(dates)

# all dates in datetime format
datesdt = []
for date in dates:
    datesdt.append(datetime.strptime(date,'%Y%m%d'))

# size of entire SLC
az0 = 0
daz= 3500
rg0 = 0
drg = 30000

# for finding the center of fields
caz = 100
crg = 1000

# coherence threshold
cohlim = .3

In [3]:
# read in field ID and road arrays
ds = gdal.Open(f'{datadir}/roadarray.r4', gdal.GA_ReadOnly)
allroadarray = ds.ReadAsArray()

ds = gdal.Open(f'{datadir}/fieldccarray.r4', gdal.GA_ReadOnly)
allfieldccarray = ds.ReadAsArray()

nf = int(np.max(allfieldccarray)) # number of fields

In [4]:
# Modified after https://github.com/opera-adt/dolphin/blob/4289be10e019da9c8da6c659370601ea96e63fe9/src/dolphin/interferogram.py#L509-L552
def moving_window_mean(image, size) -> np.ndarray:
    """Calculate the mean of a moving window of size `size`.

    Parameters
    ----------
    image : ndarray
        input image
    size : int or tuple of int
        Window size. If a single int, the window is square.
        If a tuple of (row_size, col_size), the window can be rectangular.

    Returns
    -------
    ndarray
        image the same size as `image`, where each pixel is the mean
        of the corresponding window.
    """
    if isinstance(size, int):
        size = (size, size)
    if len(size) != 2:
        raise ValueError("size must be a single int or a tuple of 2 ints")
    if size[0] % 2 == 0 or size[1] % 2 == 0:
        size = tuple(map(sum, zip(size, (1,1))))

    row_size, col_size = size
    row_pad = row_size // 2
    col_pad = col_size // 2

    # Pad the image with zeros
    image_padded = np.pad(
        image, ((row_pad + 1, row_pad), (col_pad + 1, col_pad)), mode="constant"
    )

    # Calculate the cumulative sum of the image
    integral_img = np.cumsum(np.cumsum(image_padded, axis=0), axis=1)
    if not np.iscomplexobj(integral_img):
        integral_img = integral_img.astype(float)

    # Calculate the mean of the moving window
    # Uses the algorithm from https://en.wikipedia.org/wiki/Summed-area_table
    window_mean = (
        integral_img[row_size:, col_size:]
        - integral_img[:-row_size, col_size:]
        - integral_img[row_size:, :-col_size]
        + integral_img[:-row_size, :-col_size]
    )
    window_mean /= row_size * col_size
    
    return window_mean

In [5]:
# setup empty dataframes to save
roadpixel_df = pd.DataFrame()
fieldpixel_df = pd.DataFrame()
fieldamp_df = pd.DataFrame(data=datesdt,columns=['date'])
fieldampstd_df = pd.DataFrame(data=datesdt,columns=['date'])
roadn_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
fieldn_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
roadphs_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
fieldphs_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
diffphs_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
roadskew_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
fieldskew_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
roadvar_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
fieldvar_df = pd.DataFrame(data=datesdt[1:],columns=['date'])
diffstd_df = pd.DataFrame(data=datesdt[1:],columns=['date'])

In [ ]:
# loop over each field  
for f in np.arange(1,nf+1):
    # setup empty variables for appending
    roadpixeltemp = np.array([])
    fieldpixeltemp = np.array([])
    fieldamptemp = np.array([])
    fieldampstdtemp = np.array([])
    roadntemp = np.array([])
    fieldntemp = np.array([])
    roadphstemp = np.array([])
    fieldphstemp = np.array([])
    diffphstemp = np.array([])
    roadskewtemp = np.array([])
    fieldskewtemp = np.array([])
    roadvartemp = np.array([])
    fieldvartemp = np.array([])
    diffstdtemp = np.array([])

    # center smaller array over field of interest
    fieldarray = np.zeros([daz,drg])
    fieldarray[allfieldccarray==f] = 1
    nonzero_indices = np.nonzero(fieldarray)
    
    # Calculate the center of the nonzero elements
    center = np.mean(np.array(nonzero_indices), axis=1).astype(int)
    if center[0]<caz:
        id1 = 0
    else:
        id1 = int(center[0]-caz)
    if center[0]>(daz-caz):
        id2 = int(daz)
    else:
        id2 = center[0]+caz
    if center[1]<crg:
        id3 = 0
    else:
        id3 = int(center[1]-crg)
    if center[1]>(drg-crg):
        id4 = int(drg)
    else:
        id4 = int(center[1]+crg)
    
    # crop arrays
    roadarray = allroadarray[id1:id2,id3:id4]
    fieldccarray = allfieldccarray[id1:id2,id3:id4]
    fieldarray = fieldarray[id1:id2,id3:id4]
    
    # count number of pixels within roads and field
    roadpixel = np.count_nonzero(roadarray)
    fieldpixel = np.count_nonzero(fieldarray)
    roadpixeltemp = np.append(roadpixeltemp,roadpixel)
    fieldpixeltemp = np.append(fieldpixeltemp,fieldpixel)
    
    # open first SLC
    ds1 = gdal.Open(f'{datadir}/VV/merged/SLC/{dates[0]}/{dates[0]}.slc.full', gdal.GA_ReadOnly)
    slc1 = ds1.GetRasterBand(1).ReadAsArray(id3,id1,int(id4-id3),int(id2-id1))

    # amplitude for first date in pair
    fieldmaskamp = np.zeros([int(id2-id1),int(id4-id3)],'complex')
    mask = (fieldccarray==f)
    fieldmaskamp[mask] = slc1[mask]
    fieldamp = np.mean(np.abs(fieldmaskamp[np.nonzero(np.abs(fieldmaskamp))]))
    fieldampstd = np.std(np.abs(fieldmaskamp[np.nonzero(np.abs(fieldmaskamp))]))
    fieldamptemp = np.append(fieldamptemp,fieldamp)
    fieldampstdtemp = np.append(fieldampstdtemp, fieldampstd)
    
    # loop over each sequential ifg
    for k in np.arange(nd-1):
        # open first SLC
        ds1 = gdal.Open(f'{datadir}/VV/merged/SLC/{dates[k]}/{dates[k]}.slc.full', gdal.GA_ReadOnly)
        slc1 = ds1.GetRasterBand(1).ReadAsArray(id3,id1,int(id4-id3),int(id2-id1))
        # open second SLC
        ds2 = gdal.Open(f'{datadir}/VV/merged/SLC/{dates[k+1]}/{dates[k+1]}.slc.full', gdal.GA_ReadOnly)
        slc2 = ds2.GetRasterBand(1).ReadAsArray(id3,id1,int(id4-id3),int(id2-id1))
        ifg = slc1*np.conj(slc2)

        # amplitude for second date in pair
        fieldmaskamp = np.zeros([int(id2-id1),int(id4-id3)],'complex')
        mask = (fieldccarray==f)
        fieldmaskamp[mask] = slc2[mask]
        fieldamp = np.mean(np.abs(fieldmaskamp[np.nonzero(np.abs(fieldmaskamp))]))
        fieldampstd = np.std(np.abs(fieldmaskamp[np.nonzero(np.abs(fieldmaskamp))]))
        fieldamptemp = np.append(fieldamptemp,fieldamp)
        fieldampstdtemp = np.append(fieldampstdtemp,fieldampstd)
        
        # calculate coherence
        abs1 = moving_window_mean(slc1*np.conj(slc1), 10)
        abs2 = moving_window_mean(slc2*np.conj(slc2), 10)
        coh_cpx = moving_window_mean(ifg, 10)/(np.sqrt(abs1*abs2))
        coh = np.clip(np.abs(coh_cpx), 0, 1)
        nan_mask = np.isnan(ifg)
        zero_mask = ifg == 0
        coh[nan_mask] = np.nan
        coh[zero_mask] = 0
        
        # mask to separate roads and field
        roadmask = np.zeros([int(id2-id1),int(id4-id3)],dtype='complex')
        mask = (roadarray==1)
        roadmask[mask]= ifg[mask]
        fieldmask = np.zeros([int(id2-id1),int(id4-id3)],dtype='complex')
        mask = (fieldarray==1)
        fieldmask[mask]= ifg[mask]
        
        # apply coherence threshold
        mask = (coh<cohlim)
        roadmask[mask]= 0
        fieldmask[mask]= 0
        
        # count number of coherent pixels
        roadn = np.count_nonzero(roadmask)
        fieldn = np.count_nonzero(fieldmask)
        roadntemp = np.append(roadntemp,roadn)
        fieldntemp = np.append(fieldntemp,fieldn)

        # expand array to include road pixels near road
        ckernel = cv2.getStructuringElement(cv2.MORPH_RECT,(40,8))
        dilate = cv2.dilate(fieldarray,ckernel,iterations=5)
        mask = (dilate==0)
        roadmask[mask]= 0
        fieldmask[mask]= 0
        
        # calculate average phase difference
        roadavg = np.mean(roadmask[np.nonzero(roadmask)])
        fieldavg = np.mean(fieldmask[np.nonzero(fieldmask)])
        diffphs = np.angle(fieldavg*np.conj(roadavg))
        roadphstemp = np.append(roadphstemp,np.angle(roadavg))
        fieldphstemp = np.append(fieldphstemp,np.angle(fieldavg))
        diffphstemp = np.append(diffphstemp,diffphs)

        # calculate skewness
        roadskew = scipy.stats.skew(np.angle(roadmask[np.nonzero(roadmask)]*np.conj(roadavg))) 
        fieldskew = scipy.stats.skew(np.angle(fieldmask[np.nonzero(fieldmask)]*np.conj(fieldavg))) 
        roadskewtemp = np.append(roadskewtemp,roadskew)
        fieldskewtemp = np.append(fieldskewtemp,fieldskew)

        # calculate errors
        roadvar = (-2*np.log(np.abs(roadavg)))
        fieldvar = (-2*np.log(np.abs(fieldavg)))
        diffstd = np.sqrt((roadvar/roadn)+(fieldvar/fieldn))
        roadvartemp = np.append(roadvartemp,roadvar)
        fieldvartemp = np.append(fieldvartemp,fieldvar)
        diffstdtemp = np.append(diffstdtemp,diffstd)
    # write to dataframes
    column = f'field_{f}'
    roadpixel_df[column] = roadpixeltemp
    fieldpixel_df[column] = fieldpixeltemp
    fieldamp_df[column] = fieldamptemp 
    fieldampstd_df[column] = fieldampstdtemp 
    roadn_df[column] = roadntemp 
    fieldn_df[column] = fieldntemp 
    roadphs_df[column] = roadphstemp 
    fieldphs_df[column] = fieldphstemp 
    diffphs_df[column] = diffphstemp 
    roadskew_df[column] = roadskewtemp
    fieldskew_df[column] = fieldskewtemp
    roadvar_df[column] = roadvartemp 
    fieldvar_df[column] = fieldvartemp 
    diffstd_df[column] = diffstdtemp 

In [ ]:
# save dataframes
roadpixel_df.to_pickle(f'{workdir}/dataframes/roadpixel_df.pkl')
fieldpixel_df.to_pickle(f'{workdir}/dataframes/fieldpixel_df.pkl')
fieldamp_df.to_pickle(f'{workdir}/dataframes/fieldamp_df.pkl')
fieldampstd_df.to_pickle(f'{workdir}/dataframes/fieldampstd_df.pkl')
roadn_df.to_pickle(f'{workdir}/dataframes/roadn_df.pkl')
fieldn_df.to_pickle(f'{workdir}/dataframes/fieldn_df.pkl')
roadphs_df.to_pickle(f'{workdir}/dataframes/roadphs_df.pkl')
fieldphs_df.to_pickle(f'{workdir}/dataframes/fieldphs_df.pkl')
diffphs_df.to_pickle(f'{workdir}/dataframes/diffphs_df.pkl')
roadskew_df.to_pickle(f'{workdir}/dataframes/roadskew_df.pkl')
fieldskew_df.to_pickle(f'{workdir}/dataframes/fieldskew_df.pkl')
roadvar_df.to_pickle(f'{workdir}/dataframes/roadvar_df.pkl')
fieldvar_df.to_pickle(f'{workdir}/dataframes/fieldvar_df.pkl')
diffstd_df.to_pickle(f'{workdir}/dataframes/diffstd_df.pkl')